Milestone 1 (Player detection using the YOLOv8 models trained on COCO dataset)

To install dependencies

Note: After installation, please click "RESTART RUNTIME" to restart the runtime.

In [ ]:
!pip install norfair
!pip install ultralytics

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import sys
root_path = '/content/drive/MyDrive/basketball-broadcaster'
sys.path.append(root_path)

import os
os.chdir(root_path)

import cv2
from inference.yolov8 import YOLOv8
from norfair import Video
import time


Select between the 5 YOLO8 model sizes

In [3]:
player_detector = YOLOv8('yolov8n.pt') # 'yolov8n.pt', 'yolov8s.pt', 'yolov8m.pt', 'yolov8l.pt' or 'yolov8x.pt'

Inference on a video file: the result video is outputtted in the drive root folder. For the example video below, the file "test1_clip_out.mp4" is outputted.

In [ ]:
video = Video(input_path='videos/test1_clip.mp4')

frame_cnt = 0
total_time = 0
for i, frame in enumerate(video):
    st = time.time()
    boxes, scores, class_ids = player_detector.detect(frame)
    elapsed = time.time() - st

    total_time += elapsed
    fps = frame_cnt // total_time

    frame = player_detector.draw_detection(frame, boxes, scores, class_ids)

    h, w = frame.shape[:2]
    cv2.putText(frame, 'FPS: {}'.format(fps), (100, 50), cv2.FONT_HERSHEY_SIMPLEX,
                1.0, (255, 0, 0), 2, cv2.LINE_AA)

    frame_cnt += 1
    video.write(frame)